## **Medical Report Summarisation using Medical Knowledge**

### **References**

**Main Reference**
- Radiology report generation with medical knowledge and multilevel image-report alignment: A new method and its verification
https://www.sciencedirect.com/science/article/pii/S0933365723002282#bib1



## **Data Collection**

### **Collect Datasets**

In [7]:
'''Using Kaggle API to Download Datasets - Local Machine (Windows)'''

# installing kaggle and kagglehub
!pip install kaggle
!pip install kagglehub

# kaggle.json path must be set to C:\Users\<YourUsername>\.kaggle\kaggle.json

# authenticating using kaggle api
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

ref                                                           title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                       Student Performance Factors                          94KB  2024-09-02 10:53:57          27616        485  1.0              
abdulszz/spotify-most-streamed-songs                          Spotify Most Streamed Songs                          60KB  2024-09-07 18:23:14          10453        138  1.0              
valakhorasani/mobile-device-usage-and-user-behavior-dataset   Mobile Device Usage and User Behavior Dataset        11KB  2024-09-28 20:21:12           3209         66  1.0              
thomasfranois/worlds-best-restaurants                         World's 

In [ ]:
'''Using Kaggle API to Download Datasets - Local Machine (MacOS/Linux)'''

# installing kaggle and kagglehub
!pip install kaggle
!pip install kagglehub

# kaggle.json path must be set to ~/.kaggle/kaggle.json

# authenticating using kaggle api
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
'''Using Kaggle API to Download Datasets - Google Collaboratory'''

# installing kaggle and kagglehub
!pip install kaggle
!pip install kagglehub

# kaggle.json path must be uploaded to your python notebook

# setup to use kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
# Importing the Datasets
import kagglehub

# Mimic CXR Dataset
mimic_cxr_df = kagglehub.dataset_download("wasifnafee/mimic-cxr")
print("Path to dataset files:", mimic_cxr_df)

# # IU X-Ray Dataset
iu_xray_df = kagglehub.dataset_download("raddar/chest-xrays-indiana-university")
print("Path to dataset files:", iu_xray_df)

100%|██████████| 8.95M/8.95M [00:00<00:00, 41.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/wasifnafee/mimic-cxr/versions/1


100%|██████████| 13.2G/13.2G [03:06<00:00, 75.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2


## **Data Preprocessing**

### **Preprocess Images**

### **Preprocess Text**

### **Create Data Loaders**

## **Model Implementation**

### **Visual Extractor**

### **Text Encoder**

### **Multilevel Alignment**

### **Report Generator**

### **Complete Model**

## **Training**

### **Training**

## **Testing**

### **Testing**